In [25]:
import os
import re
import yfinance as yf
import pandas as pd
from datetime import datetime

# 종목명 ↔ Yahoo Finance 코드 매핑
symbol_map = {
    "삼성전자": "005930.KS",
    "LG전자": "066570.KS",
    "카카오": "035720.KQ",
    "현대차": "005380.KS",
    "SK하이닉스": "000660.KS",
    "기아": "000270.KS",
    "NAVER": "035420.KQ",
    "삼성바이오로직스": "207940.KS",
    "셀트리온": "068270.KQ",
    "LG화학": "051910.KS",
    "삼성SDI": "006400.KS"
}

# 자연어 입력 → 종목명, 시작일, 종료일 추출
def parse_input(text):
    name_match = re.search(r"(" + "|".join(symbol_map.keys()) + r")", text)
    year_match = re.search(r"(\d{4})년", text)
    month_match = re.search(r"(\d{1,2})월", text)

    if not (name_match and year_match and month_match):
        raise ValueError("⚠️ 문장에서 종목명, 연도, 월을 모두 인식할 수 없습니다.")

    stock_name = name_match.group(1)
    year = int(year_match.group(1))
    month = int(month_match.group(1))

    symbol = symbol_map[stock_name]
    start_date = f"{year}-{month:02d}-01"
    if month == 12:
        end_date = f"{year+1}-01-01"
    else:
        end_date = f"{year}-{month+1:02d}-01"

    return stock_name, symbol, start_date, end_date

# 주가 데이터 가져오기 및 요약
def fetch_stock_data(symbol: str, start_date: str, end_date: str) -> dict:
    import yfinance as yf
    import pandas as pd

    data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=False, progress=False)

    if data.empty:
        return {"error": "해당 기간에 대한 주가 데이터가 없습니다."}

    # 멀티인덱스일 경우 첫 번째 레벨 제거
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.droplevel(1)

    return {
        "평균 종가": round(data["Close"].mean(), 2),
        "최고 종가": round(data["Close"].max(), 2),
        "최저 종가": round(data["Close"].min(), 2),
        "총 거래일 수": len(data),
        "최근 5일": data[["Open", "High", "Low", "Close", "Volume"]].tail().to_dict(orient="index")
    }



# 🔁 실행 부분
if __name__ == "__main__":
    user_input = input("▶ 주가를 확인하고 싶은 문장을 입력하세요:\n> ")

    try:
        stock_name, symbol, start, end = parse_input(user_input)
        result = fetch_stock_data(symbol, start, end)

        if "error" in result:
            print(f"❗ {result['error']}")
        else:
            print(f"\n📈 [{stock_name}] {start} ~ {end} 주가 요약:")
            print(f"- 평균 종가: {result['평균 종가']:,}원")
            print(f"- 최고 종가: {result['최고 종가']:,}원")
            print(f"- 최저 종가: {result['최저 종가']:,}원")
            print(f"- 총 거래일 수: {result['총 거래일 수']}일")

            print("\n📅 최근 5일 종가 요약:")
            for date, values in result["최근 5일"].items():
                d = date.strftime("%Y-%m-%d")
                open_ = values["Open"]
                high = values["High"]
                low = values["Low"]
                close = values["Close"]
                print(f"🗓 {d} | 시가: {open_:,.0f} | 고가: {high:,.0f} | 저가: {low:,.0f} | 종가: {close:,.0f}")

    except Exception as e:
        print(f"[오류] {e}")



📈 [기아] 2024-03-01 ~ 2024-04-01 주가 요약:
- 평균 종가: 120,735.0원
- 최고 종가: 128,500.0원
- 최저 종가: 110,200.0원
- 총 거래일 수: 20일

📅 최근 5일 종가 요약:
🗓 2024-03-25 | 시가: 113,100 | 고가: 113,500 | 저가: 111,000 | 종가: 111,500
🗓 2024-03-26 | 시가: 111,200 | 고가: 115,400 | 저가: 111,200 | 종가: 114,400
🗓 2024-03-27 | 시가: 114,400 | 고가: 115,000 | 저가: 112,900 | 종가: 113,600
🗓 2024-03-28 | 시가: 114,200 | 고가: 114,300 | 저가: 111,800 | 종가: 112,000
🗓 2024-03-29 | 시가: 112,200 | 고가: 113,400 | 저가: 110,000 | 종가: 110,200
